In [1]:
from __future__ import division
import graphlab
import math
import string
import numpy as np

ImportError: No module named graphlab

In [2]:
products = graphlab.SFrame('tweetclassifier1_13csv.csv')

def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['text'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)
products = products[products['sentiment'] != 0]

#print '# of positive reviews =', len(products[products['sentiment']==1])
#print '# of negative reviews =', len(products[products['sentiment']==-1])

train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    correct_examples = []
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    for i in range(0,len(predictions)):
        if predictions[i] != true_labels[i]: continue
        else:
            correct_examples.append(predictions[i])
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = len(correct_examples)/len(predictions)
    
    return accuracy

AttributeError: 'module' object has no attribute 'SFrame'

In [44]:
significant_words = ['bullish', 'bearish', 'rally', 'bears', 'bulls', 'bull', 'bear', 'decline', 
                    'rise', 'support', 'resistance', 'higher', 'lower']

train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

print '# of positive reviews =', len(test_data[test_data['sentiment']==1])
print '# of negative reviews =', len(test_data[test_data['sentiment']==-1])

#test_data = test_data[test_data['word_count_subset'] != {}]
#train_data = train_data[train_data['word_count_subset'] != {}]


simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

get_classification_accuracy(simple_model, test_data, test_data['sentiment'])
simple_model.coefficients
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

# of positive reviews = 101
# of negative reviews = 76


Logistic regression:

--------------------------------------------------------

Number of examples          : 781

Number of classes           : 2

...

| 2         | 3        | 0.004004     | 0.650448          |

| 3         | 4        | 0.006006     | 0.650448          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

+-------------------+------------+-------+-----------------+----------------+
|        name       |   index    | class |      value      |     stderr     |
+-------------------+------------+-------+-----------------+----------------+
| word_count_subset |    bull    |   1   |  0.895578731796 | 0.467227114233 |
| word_count_subset |   bears    |   1   |  0.699800350602 | 0.367294254146 |
| word_count_subset |    rise    |   1   |  0.600926205321 | 0.381322070839 |
| word_count_subset | resistance |   1   |  0.565388747883 | 0.438566399202 |
| word_count_subset |   bulls    |   1   |  0.524241363912 | 0.48509505703  |
| word_count_subset |  bullish   |   1   |  0.404954311586 | 0.235019350381 |
| word_count_subset |    bear    |   1   |  0.358270581171 | 0.378234348215 |
| word_count_subset |   rally    |   1   |  0.342079041239 | 0.285565606358 |
| word_count_subset |  decline   |   1   |  0.296166966125 | 0.59998695973  |
|    (intercept)    |    None    |   1   | 0.0851344775123 | 0.2

In [39]:
scores = simple_model.predict(test_data, output_type='margin')
print scores
ytag = np.sign(scores)
print ytag

[-0.6259195560304072, 0.42721351875110486, 0.4900887890984375, 0.6505232253953458, 0.6505232253953458, 0.4900887890984375, 0.4900887890984375, 0.42721351875110486, 0.6505232253953458, 0.4434050586829224, 0.3813014436368259, -0.6259195560304072, 0.686060682833728, 0.6505232253953458, 0.6505232253953458, -0.02900378543722755, -0.6259195560304072, -0.6259195560304072, -0.02900378543722755, 0.42721351875110486, 0.4900887890984375, 0.42721351875110486, 0.4900887890984375, 0.4900887890984375, -0.4339580970233631, 0.4900887890984375, 0.4900887890984375, -0.4339580970233631, 0.4900887890984375, 0.6505232253953458, -0.6259195560304072, 0.9807132093087724, -0.6259195560304072, -0.6259195560304072, 0.4900887890984375, -0.6259195560304072, -0.6259195560304072, -0.6259195560304072, -0.6259195560304072, -0.6259195560304072, 0.9807132093087724, -0.6259195560304072, 0.42721351875110486, 0.4434050586829224, -0.6259195560304072, 0.9807132093087724, 0.4900887890984375, 0.4434050586829224, 0.4434050586829

In [40]:
test_data['probability_prediction']=simple_model.predict(test_data, output_type='probability')
top20 = test_data.topk('probability_prediction', k=20)
#print top20.column_names()
print top20[0:20]

+----------------+-------------------------------+
| relative_index |              text             |
+----------------+-------------------------------+
|     11144      | #usda bears are happy, bul... |
|     26049      | I think it's about time to... |
|     12684      | @glen_newcomer At first gl... |
|     30112      | RT @KelseyStanger: "I will... |
|      9965      |  Two corn dogs and a red bull |
|     25298      | Tom &amp; jerry + pop corn... |
|     25294      |   @ktbug51 your bull corn ??  |
|     29133      | RT @Agrimoney: Soybean fut... |
|     10977      | RT @NaijaNupe_: Doesn't ea... |
|      8928      | RT @ShawneeState: Lots of ... |
+----------------+-------------------------------+
+--------------------------------+--------------+-----------+
|              time              | prct_change  | sentiment |
+--------------------------------+--------------+-----------+
| Fri Aug 12 16:17:14 +0000 2016 | -0.313479624 |     -1    |
| Fri Aug 12 23:42:14 +0000 2016 | 1.1

In [41]:
print top20[2]['text']

@glen_newcomer At first glance, looked like you were burying someone in that corn field. A grave big enough for the last corn bull.
